DOING ALL THE IMPORTS

In [25]:
from dotenv import load_dotenv
import os 
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

ADDING LLM

In [3]:
load_dotenv()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature = 0,
    api_key = os.getenv("GOOGLE_API_KEY")
)

Loading pdf and extracting text for embedding

In [ ]:
pdf_path = r'C:\AYUSH\comet\Comet_AI_Assistance\AyushSingh_AIMl_Resume_2026_compressed.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()
#add in text 
text = ""
for doc in docs:
    text += doc.page_content + "\n"
print(text)


Ayush Singh 
ayushrajput8252@gmail.com | +917209602122 | linkedin.com/in/ayush-singh 
 
Summary 
GenAI / AI-ML Engineer specializing in RAG pipelines, LLM applications, LangChain/LangGraph workflows, and 
AI Agents. Built production-style RAG systems, OCR-based verification automation, and transfer learning models 
using Python, vector databases, FastAPI, and Docker. Hackathon winner (Top 10/1000 teams) with strong 
problem-solving background (1000+ DSA problems solved). 
Education 
Parul University, B.Tech in Computer Science 2022 – 2026 
Experience 
Capgemini, Pre-Joining Technical Training Program Jan 2026 – Present 
• Selected through campus recruitment and received an Offer of Intent (LOI). 
• Currently undergoing structured pre-joining training in DSA, OOP, debugging, and coding 
best practices. 
• Solving company-style coding assignments with focus on edge cases and time complexity. 
Skills 
Languages : Java, C++, Python                                                           

TEXT EXTRACTION BASED ON HEADING

In [48]:
import re

HEADINGS = [
    "SUMMARY", "OBJECTIVE",
    "SKILLS", "TECHNICAL SKILLS",
    "EDUCATION",
    "EXPERIENCE", "WORK EXPERIENCE",
    "PROJECTS", "PROJECT",
    "CERTIFICATIONS",
    "ACHIEVEMENTS",
    "INTERNSHIPS",
    "LANGUAGES"
]
REQUIRED_WORDS = {
    "github",
    "gmail.com",
    "linkedin",
    "leetcode"
}
important = []
def check_required_words(text):
    text = text.lower()
    for word in REQUIRED_WORDS:
        if word not in text:
            important.append(word)

def split_heading(text):
    sections = {"OTHER" : ""}
    current = "OTHER"
   
    for line in text.split("\n"):
        line = line.strip()

        if not line :
            continue

        normalized = re.sub(r"[^A-Za-z]", "", line).upper().strip()

        if normalized in HEADINGS:
            current = normalized
            sections[current] = ""
        else:
            sections[current] += line + "\n"

    return sections

INITIAL BASIC CHECKS

In [ ]:
sections = split_heading(text)
missing = []
count = 0

if sections.get("SKILLS") or sections.get("TECHNICAL SKILLS"):
    count += 1
else:
    missing.append("SKILLS")

if sections.get("SUMMARY") or sections.get("OBJECTIVE"):
    count += 1
else:
    missing.append("SUMMARY")

if sections.get("EXPERIENCE") or sections.get("WORK EXPERIENCE"):
    count += 1
else:
    missing.append("EXPERIENCE")

if sections.get("INTERNSHIPS"):
    count += 1
else:
    missing.append("INTERNSHIPS")

if sections.get("PROJECTS") or sections.get("PROJECT"):
    count += 1
else :
    missing.append("PROJECTS")

if count >= 4:
    print("Your Resume Contains All necessary info")
else:
    print("field missing:")
    print(5 - len(missing))
    for show in missing:
        print(show)


Your Resume Contains All necessary info


Resume improvement suggestions
 1. does resume contains achivements
 2. does resume contains certificates 

In [49]:
if not sections.get("ACHIEVEMENTS"):
    print("ADD ACHIEVEMENTS")

if not sections.get("CERTIFICATIONS"):
    print("ADD CERTIFICATIONS")

for imp in important:
    print("missing")
    print(imp)


ADD CERTIFICATIONS


SENDING SUMMARY AND SKILLS AND TELLING IF THEY ALLIGNS OR NOT

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# 1) title from summary
prompt1 = ChatPromptTemplate.from_template(
    "Act like a professional IT recruiter. Read this summary:\n{summary}\n"
    "Give exactly one job title that matches it. "
    "Output must be lowercase and max 2 words."
)

chain1 = prompt1 | llm | parser
result1 = chain1.invoke({"summary": sections.get("SUMMARY", "")})

# 2) title from skills
prompt2 = ChatPromptTemplate.from_template(
    "Act like a professional IT recruiter. Read these skills:\n{skills}\n"
    "Give exactly one job title that matches it. "
    "Output must be lowercase and max 2 words."
)

chain2 = prompt2 | llm | parser
result2 = chain2.invoke({"skills": sections.get("SKILLS", "")})

print("Summary Title:", result1)
print("Skills Title:", result2)


Summary Title: genai engineer
Skills Title: ai engineer


In [32]:
prompt3 = ChatPromptTemplate.from_template(
    "Compare these two job titles:\n"
    "1) {t1}\n"
    "2) {t2}\n\n"
    "Return similarity score between 0 and 1 (only number)."
)
chain3 = prompt3 | llm | parser
score = float(chain3.invoke({"t1" : result1, "t2": result2}))
print(score)
if score > 0.8:
    print("GOOD")
else:
    print("Change skill and summary")


0.95
GOOD


Now do a web search through duckduck go or travily extract insignt about this
1. this job tiltle
2. question not genric but from interview experiences 
3. user get 2 option 1. free to chat 2. live vc which give marks on the basis of voice modulalation, clarity, how good he spoke, sound, relevancy and more

In [38]:
from tavily import TavilyClient
api_key = os.getenv("TAVILY_API_KEY")
load_dotenv() 
client = TavilyClient(api_key=api_key)
#here result2 is from skills
title = result2
query = f"{title} placement roadmap trends for 2026"

response = client.search(
    query = query,
    search_depth="advanced",
    max_results = 10,
    include_answer=True,
    include_raw_content=True
)
print(response["answer"])
for r in response["results"]:
    print("\n======================")
    print("TITLE:", r["title"])
    print("URL:", r["url"])
    print("CONTENT:\n", r["content"])

To become an AI engineer in 2026, focus on mastering Python, machine learning, and practical projects. Build a strong portfolio and network with industry professionals. Stay updated with the latest AI trends and technologies.

TITLE: AI Engineer Roadmap 2026: Own the Future of AI - LinkedIn
URL: https://www.linkedin.com/pulse/ai-engineer-roadmap-2026-own-future-careerxperts-consulting-eburc
CONTENT:
 ### Step Into Ownership

AI engineering isn’t for spectators. It’s for people who step in, take responsibility, and deliver. Follow this roadmap with discipline, apply relentlessly, and career trajectories will accelerate-especially in startup hiring, where impact and ownership matter most.

The landscape is crowded. But there’s a difference between those who watch trends and those who define them. Own learning, own projects, own career.

For candidates aiming to land AI engineer roles, this is the moment to showcase skills through real projects, GenAI mastery, and cloud deployment experie

give me context and tell to summaries and get placement tips

In [53]:

prompt = ChatPromptTemplate.from_template(
    "This is a roadmap for placement. Go through the context and give short tips in bullet points.\n\nContext:\n{context}"
)
parser = StrOutputParser()
chain4 = prompt | llm | parser
result3 = chain4.invoke({"context" : response["answer"]})
print(result3)

Here are short tips based on your roadmap:

*   **Master Python:** Become highly proficient in Python programming.
*   **Master Machine Learning:** Develop a deep understanding of ML concepts and algorithms.
*   **Build Practical Projects:** Gain hands-on experience through real-world AI projects.
*   **Cultivate a Strong Portfolio:** Showcase your best projects and skills effectively.
*   **Network Actively:** Connect with professionals in the AI industry.
*   **Stay Updated:** Continuously learn about new AI trends and technologies.


Extract questions and answers

In [62]:
print(result2)

ai engineer


In [ ]:
import os
from dotenv import load_dotenv
from tavily import TavilyClient

load_dotenv()

api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

title = result2
all_links = []
#doing the loops to get 
queries = [
    f'site:geeksforgeeks.org "{title}" "interview experience"',
    f'site:leetcode.com "{title}" interview questions',
    f'site:interviewbit.com "{title}" interview questions',
    f'site:medium.com "{title}" interview questions',
    f'site:glassdoor.co.in "{title}" interview questions'
]

for q in queries:
    print("\nSEARCHING:", q)
    link = client.search(
        query=q,
        search_depth = "advanced",
        max_results = 3,
        include_answer=False,
        include_raw_content=False
    )
    #stores links only in all_links
    for r in link["results"]:
        all_links.append(r["url"])
        print("TITLE:", r["title"])
        print("URL:", r["url"])
print("\n\nTOTAL LINKS FOUND:", len(all_links))


SEARCHING: site:geeksforgeeks.org "ai engineer" "interview experience"
TITLE: Amazon AI Engineer Interview Experience - GeeksforGeeks
URL: https://www.geeksforgeeks.org/interview-experiences/amazon-ai-engineer-interview-experience/
TITLE: LinkedIn Interview Experience for AI Intern - GeeksforGeeks
URL: https://www.geeksforgeeks.org/interview-experiences/linkedin-interview-experience-for-ai-intern/
TITLE: How to Land an Artificial Intelligence Internship in 2024
URL: https://www.geeksforgeeks.org/artificial-intelligence/how-to-land-an-artificial-intelligence-internship/

SEARCHING: site:leetcode.com "ai engineer" interview questions
TITLE: Linkedin | Software Engineer, Machine Learning Interview Experience
URL: https://leetcode.com/discuss/post/7405614/linkedin-software-engineer-machine-learn-l9pd/
TITLE: LinkedIn Senior AI engineer, Bay Area (Rejected) - Discuss
URL: https://leetcode.com/discuss/interview-experience/5976536/LinkedIn-Senior-AI-engineer-(Rejected)/
TITLE: Top Interview 

✅ Save links in JSON

In [68]:
import os
import json

os.makedirs("data", exist_ok=True)

with open("data/links.json", "w") as file:
    json.dump(all_links, file, indent = 4)

print("Links saved in json")


Links saved in json


✅ Read links later
import json

with open("data/links.json", "r") as file:
    all_links = json.load(file)

print(all_links)